In [117]:
# PART I.
# using numerical method to approximate phi(x)

import math

def CompSimpsonf1(a,b,n):
    h = (b-a)/n
    XI0 = f1(a)+f1(b)
    XI1 = 0
    XI2 = 0
    for i in range(1,n):
        X = a+i*h
        if i%2 == 0:
            XI2 += f1(X)
        else:
            XI1 += f1(X)
    XI = h/3*(XI0+2*XI2+4*XI1)
    return XI
def CompSimpsonf2(a,b,n):
    h = (b-a)/n
    XI0 = f2(a)+f2(b)
    XI1 = 0
    XI2 = 0
    for i in range(1,n):
        X = a+i*h
        if i%2 == 0:
            XI2 += f2(X)
        else:
            XI1 += f2(X)
    XI = h/3*(XI0+2*XI2+4*XI1)
    return XI

def CompMid1(a,b,n):
    h = (b-a)/(n+2)
    XI1 = 0
    for i in range(1,n+2):
        if i%2 != 0:
            X = a+i*h
            XI1 += f1(X)
    XI = 2*h*XI1
    return XI
def CompMid2(a,b,n):
    h = (b-a)/(n+2)
    XI1 = 0
    for i in range(1,n+2):
        if i%2 != 0:
            X = a+i*h
            XI1 += f2(X)
    XI = 2*h*XI1
    return XI

def f1(x):
    return math.exp(-1/(2*x**2))/x**2
def f2(x):
    return math.exp(-x**2/2)

#  use Composite Midpoint Rule instead of Composite Simpson's rule because f1(x) is not defined on x=0, 
#  thus we need to do Open Newton-Cotes Formulas
def phi(x):
    return (1/math.sqrt(2*math.pi))*(CompMid1(-1,0,100)+CompMid2(-1,x,100))
phi(100)

1.0623579283658153

In [118]:
# an attempt to use Gaussian Quadrature for this problem

def f1(x):
    return math.exp(-1/(2*x**2))/(x**2)
def f2(x):
    return math.exp(-x**2/2)
def transformf1(a,b,x):
    return f1(((b-a)*x+b+a)/2)*(b-a)/2
def GaussianN51(a,b):
    return 0.2369268850*transformf1(a,b,0.9061798459)+\
0.4786286705*transformf1(a,b,0.5384693101)+\
0.5688888889*transformf1(a,b,0)+0.4786286705*transformf1(a,b,-0.5384693101)\
+0.2369268850*transformf1(a,b,-0.9061798459)
def transformf2(a,b,x):
    return f2(((b-a)*x+b+a)/2)*(b-a)/2
def GaussianN52(a,b):
    return 0.2369268850*transformf2(a,b,0.9061798459)+\
0.4786286705*transformf2(a,b,0.5384693101)+\
0.5688888889*transformf2(a,b,0)+0.4786286705*transformf2(a,b,-0.5384693101)\
+0.2369268850*transformf2(a,b,-0.9061798459)

def phi1(x):
    return (1/math.sqrt(2*math.pi))*(GaussianN51(-1,0)+GaussianN52(-1,x))
phi1(100)
# not using Gaussian Quadrature because it is hard to realize large n, 
# which means when the domain is big, the result could be very inaccurate

0.16530592459800603

In [312]:
# PART II.

import numpy as np
K = 40
S0 = 36
r = 0.08
vol = 0.3
div = 0
T = 1
Smax = 80
N = 100
M = 100
delta_t = T/N
delta_S = Smax/M

def jacobi(A,b,N,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                                                                                                                            
    if x is None:
        x = zeros(len(A[0]))
    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A                                                                                                                                                                     
    D = diag(A)
    R = A - diagflat(D)
    # Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x

initial = np.zeros(99)

def Ci(A,b,N,x):
    
    # represent A
    A = np.zeros((99,99))
    for a in range(1,98):
        alpha = (delta_t/2)*(r*(a+1)-vol**2*(a+1)**2)
        beta = 1+delta_t*(vol**2*(a+1)**2+r)
        gamma = -(delta_t/2)*(vol**2*(a+1)**2+r*(a+1))
        A[a][a-1:a+2] = [alpha, beta, gamma]
    beta1 = 1+delta_t*(vol**2*(1)**2+r)
    gamma1 = -(delta_t/2)*(vol**2*(1)**2+r*(1))
    A[0][0:2] = [beta1,gamma1]
    alpha99 = (delta_t/2)*(r*(99)-vol**2*(99))
    beta99 = 1+delta_t*(vol**2*(99)**2+r)
    A[98][-2] = alpha99
    A[98][-1] = beta99
    
    i = 100
    # represent C100
    Cn = np.zeros(99)
    for j in range(1,100):
        profit = (j)*delta_S-K
        if profit < 0 :
            Cn[j-1] = 0
        if profit >= 0:
            Cn[j-1] = profit
    
    # represent Bi
    while i>0:    
        B = np.zeros(99) 
        alpha1 = (delta_t/2)*(r*1-vol**2*1)
        B[0] = alpha1 * 0 # boundary condition when S = 0
        gamma99 = -(delta_t/2)*(vol**2*99**2+r*99)
        B[-1] = gamma99 * (Smax-K*math.exp(-r*(T-i*delta_t)))
        
        initial = [1]*99
        b = Cn-B
        Cn_1 = jacobi(A, b, 10000, initial)
        Cn = Cn_1
        i = i-1
    return Cn


In [304]:
import numpy as np
from numpy import array, zeros, diag, diagflat, dot
K = 40
S0 = 36
r = 0.08
vol = 0.3
div = 0
T = 1
Smax = 80
N = 100
M = 100
delta_t = T/N
delta_S = Smax/M

def jacobi(A,b,N,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                                                                                                                            
    if x is None:
        x = zeros(len(A[0]))
    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A                                                                                                                                                                     
    D = diag(A)
    R = A - diagflat(D)
    # Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x


Cn = np.zeros(99)
B = np.zeros(99)
for i in range(1,100):
    # represent Cn
    profit = (i)*0.8-K*math.exp(-r*(N-i)*delta_t)
    if profit < 0 :
        Cn[i-1] = 0
    if profit >= 0:
        Cn[i-1] = profit
    # represent Bi
    alpha1 = (delta_t/2)*(r*1-vol**2*1)
    B[0] = alpha1 * 0 # boundary condition when S = 0
    gamma99 = -(delta_t/2)*(vol**2*99**2+r*99)
    B[-1] = gamma99 * (Smax-K*math.exp(-r*(T-i*delta_t)))

# represent A
A = np.zeros((99,99))
for j in range(1,98):
    alpha = (delta_t/2)*(r*(j+1)-vol**2*(j+1)**2)
    beta = 1+delta_t*(vol**2*(j+1)**2+r)
    gamma = -(delta_t/2)*(vol**2*(j+1)**2+r*(j+1))
    A[j][j-1:j+2] = [alpha, beta, gamma]
beta1 = 1+delta_t*(vol**2*(1)**2+r)
gamma1 = -(delta_t/2)*(vol**2*(1)**2+r*(1))
A[0][0:2] = [beta1,gamma1]
alpha99 = (delta_t/2)*(r*(99)-vol**2*(99))
beta99 = 1+delta_t*(vol**2*(99)**2+r)
A[98][-2] = alpha99
A[98][-1] = beta99

initial = [1]*99
b = Cn-B

jacobi(A, b, 10000, initial)

array([1.31125345e-46, 1.54527362e-43, 5.96950583e-41, 1.14716008e-38,
       1.32336759e-36, 1.02195920e-34, 5.67583668e-33, 2.38580059e-31,
       7.88649480e-30, 2.11233261e-28, 4.69571214e-27, 8.83646481e-26,
       1.43111574e-24, 2.02298179e-23, 2.52626134e-22, 2.81639289e-21,
       2.82899441e-20, 2.58119621e-19, 2.15472426e-18, 1.65630587e-17,
       1.17917009e-16, 7.81555688e-16, 4.84546308e-15, 2.82200421e-14,
       1.54994272e-13, 8.05657886e-13, 3.97624715e-12, 1.86886497e-11,
       8.38791916e-11, 3.60411547e-10, 1.48601608e-09, 5.89202954e-09,
       2.25107736e-08, 8.30242963e-08, 2.96113218e-07, 1.02292813e-06,
       3.42781234e-06, 1.11578452e-05, 3.53266426e-05, 1.08921677e-04,
       3.27426731e-04, 9.60658713e-04, 2.75371138e-03, 7.71924957e-03,
       2.11801051e-02, 5.69303965e-02, 1.50027000e-01, 3.87909691e-01,
       9.56494589e-01, 1.64854283e+00, 2.38754069e+00, 3.14470603e+00,
       3.90901505e+00, 4.67617152e+00, 5.44447275e+00, 6.21323291e+00,
      

In [311]:
# represent C100
Cn = np.zeros(99)
for j in range(1,100):
    profit = (j)*0.8-K
    if profit < 0 :
        Cn[j-1] = 0
    if profit >= 0:
        Cn[j-1] = profit
Cn

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.8,  1.6,  2.4,  3.2,  4. ,
        4.8,  5.6,  6.4,  7.2,  8. ,  8.8,  9.6, 10.4, 11.2, 12. , 12.8,
       13.6, 14.4, 15.2, 16. , 16.8, 17.6, 18.4, 19.2, 20. , 20.8, 21.6,
       22.4, 23.2, 24. , 24.8, 25.6, 26.4, 27.2, 28. , 28.8, 29.6, 30.4,
       31.2, 32. , 32.8, 33.6, 34.4, 35.2, 36. , 36.8, 37.6, 38.4, 39.2])

In [235]:
A = np.zeros((99,99))
for i in range(1,98):
    alpha = (delta_t/2)*(r*(i+1)-vol**2*(i+1))
    beta = 1+delta_t*(vol**2*(i+1)**2+r)
    gamma = -(delta_t/2)*(vol**2*(i+1)**2+r*(i+1))
    A[i][i-1:i+2] = [alpha, beta, gamma]
beta1 = 1+delta_t*(vol**2*(1)**2+r)
gamma1 = -(delta_t/2)*(vol**2*(1)**2+r*(1))
A[0][0:2] = [beta1,gamma1]
alpha99 = (delta_t/2)*(r*(99)-vol**2*(99))
beta99 = 1+delta_t*(vol**2*(99)**2+r)
A[98][-3:-1] = [alpha99,beta99]
A

array([[ 1.00170e+00, -8.50000e-04,  0.00000e+00, ...,  0.00000e+00,
         0.00000e+00,  0.00000e+00],
       [-1.00000e-04,  1.00440e+00, -2.60000e-03, ...,  0.00000e+00,
         0.00000e+00,  0.00000e+00],
       [ 0.00000e+00, -1.50000e-04,  1.00890e+00, ...,  0.00000e+00,
         0.00000e+00,  0.00000e+00],
       ...,
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, ...,  9.46890e+00,
        -4.27285e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, ..., -4.90000e-03,
         9.64440e+00, -4.36100e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, ..., -4.95000e-03,
         9.82170e+00,  0.00000e+00]])

In [187]:
np.array([[1,2,3,4,5]]*10)

array([[1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5],
       [1, 2, 3, 4, 5]])

In [224]:
A = np.zeros(10)
A[0:2] = [1,2]
A

array([1., 2., 0., 0., 0., 0., 0., 0., 0., 0.])

In [316]:
# represent A
A = np.zeros((99,99))
for a in range(1,98):
    alpha = (delta_t/2)*(r*(a+1)-vol**2*(a+1)**2)
    beta = 1+delta_t*(vol**2*(a+1)**2+r)
    gamma = -(delta_t/2)*(vol**2*(a+1)**2+r*(a+1))
    A[a][a-1:a+2] = [alpha, beta, gamma]
beta1 = 1+delta_t*(vol**2*(1)**2+r)
gamma1 = -(delta_t/2)*(vol**2*(1)**2+r*(1))
A[0][0:2] = [beta1,gamma1]
alpha99 = (delta_t/2)*(r*(99)-vol**2*(99))
beta99 = 1+delta_t*(vol**2*(99)**2+r)
A[98][-2] = alpha99
A[98][-1] = beta99
    

# represent C100
Cn = np.zeros(99)
for j in range(1,100):
    profit = (j)*delta_S-K
    if profit < 0 :
        Cn[j-1] = 0
    if profit >= 0:
        Cn[j-1] = profit

i = 100
# represent Bi
while i>0:    
    B = np.zeros(99) 
    alpha1 = (delta_t/2)*(r*1-vol**2*1)
    B[0] = alpha1 * 0 # boundary condition when S = 0
    gamma99 = -(delta_t/2)*(vol**2*99**2+r*99)
    B[-1] = gamma99 * (Smax-K*math.exp(-r*(T-i*delta_t)))
        
    initial = [1]*99
    b = Cn-B
    Cn_1 = jacobi(A, b, 100, initial)
    Cn = Cn_1
    i = i-1
Cn[44]

3.6761634645140915